In [1]:
# Librerie
from pyspark.sql import SparkSession
from pyspark.sql.window import Window as W
import pyspark.sql.types as T
import pyspark.sql.functions as F

In [2]:
# Creazione Spark Session
spark = (SparkSession.builder.appName("DipendentiAnalytics").getOrCreate())

bash: /opt/conda/lib/libtinfo.so.6: no version information available (required by bash)
bash: /opt/conda/lib/libtinfo.so.6: no version information available (required by bash)
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/02/23 10:33:49 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
# Definizione schema dati
schema = T.StructType([
    T.StructField("Nome", T.StringType(), True),
    T.StructField("Cognome", T.StringType(), True),
    T.StructField("Reparto", T.StringType(), True),
    T.StructField("Stipendio", T.IntegerType(), True)
])

In [4]:
# Percorso del csv file che vogliamo leggere
dipendenti_fpath = "/kaggle/input/dipendenti/Dipendenti.csv"

# Lettura del csv file
dipendenti_data_raw = (spark.read.format("csv")
                       .option("header", True)
                       .option("delimiter", ";")
                       .schema(schema)
                       .load(dipendenti_fpath))

# Stampare lo schema
dipendenti_data_raw.printSchema()

root
 |-- Nome: string (nullable = true)
 |-- Cognome: string (nullable = true)
 |-- Reparto: string (nullable = true)
 |-- Stipendio: integer (nullable = true)



In [5]:
dipendenti_data_raw.show(10)

+--------+--------+-------+---------+
|    Nome| Cognome|Reparto|Stipendio|
+--------+--------+-------+---------+
|  Andrea|  Assini|     IT|     1500|
|   Bruno| Bettini|     IT|     2000|
|   Carlo|Cicerini|     IT|     1800|
|   Dario|  Desini|     HR|     1200|
|   Elena| Ettorre|     HR|     1600|
|Federica| Fierini|     HR|     2200|
|  Giulia|Giorgini|     HR|     1400|
|   Ivana|  Itteri|    CEO|     3000|
+--------+--------+-------+---------+



In [6]:
# Selezionare nome e cognome
dipendenti_data_raw.select("Nome", "Cognome").show(10)

+--------+--------+
|    Nome| Cognome|
+--------+--------+
|  Andrea|  Assini|
|   Bruno| Bettini|
|   Carlo|Cicerini|
|   Dario|  Desini|
|   Elena| Ettorre|
|Federica| Fierini|
|  Giulia|Giorgini|
|   Ivana|  Itteri|
+--------+--------+



In [7]:
# Aggiungere la colonna NomeCognome
dipendenti_data_raw = dipendenti_data_raw.withColumn("NomeCognome", F.concat(F.col("Nome"), F.col("Cognome")))
dipendenti_data_raw.show(10)

+--------+--------+-------+---------+---------------+
|    Nome| Cognome|Reparto|Stipendio|    NomeCognome|
+--------+--------+-------+---------+---------------+
|  Andrea|  Assini|     IT|     1500|   AndreaAssini|
|   Bruno| Bettini|     IT|     2000|   BrunoBettini|
|   Carlo|Cicerini|     IT|     1800|  CarloCicerini|
|   Dario|  Desini|     HR|     1200|    DarioDesini|
|   Elena| Ettorre|     HR|     1600|   ElenaEttorre|
|Federica| Fierini|     HR|     2200|FedericaFierini|
|  Giulia|Giorgini|     HR|     1400| GiuliaGiorgini|
|   Ivana|  Itteri|    CEO|     3000|    IvanaItteri|
+--------+--------+-------+---------+---------------+



In [8]:
# Selezionare solo i dipendenti con uno stipendio > 2000
dipendenti_data_raw.filter(F.col("stipendio")>2000).drop("NomeCognome").show(10)

+--------+-------+-------+---------+
|    Nome|Cognome|Reparto|Stipendio|
+--------+-------+-------+---------+
|Federica|Fierini|     HR|     2200|
|   Ivana| Itteri|    CEO|     3000|
+--------+-------+-------+---------+



In [9]:
# Calcolare il numero di dipendente
numero_dipendenti = dipendenti_data_raw.count()
numero_dipendenti

8

In [10]:
# Calcolare stipendio medio per reparto
stipendio_medio_df = dipendenti_data_raw.groupBy("Reparto").agg(F.avg("Stipendio").alias("Stipendio Medio"))
stipendio_medio_df = stipendio_medio_df.withColumn("Stipendio Medio", F.format_number("Stipendio Medio", 2))
stipendio_medio_df.show(10)

+-------+---------------+
|Reparto|Stipendio Medio|
+-------+---------------+
|     HR|       1,600.00|
|    CEO|       3,000.00|
|     IT|       1,766.67|
+-------+---------------+



In [11]:
# Calcolare la somma di tutti gli stipendi
totale_stipendi = dipendenti_data_raw.groupBy().agg(F.sum("Stipendio").alias("Totale stipendi"))
totale_stipendi.show()

+---------------+
|Totale stipendi|
+---------------+
|          14700|
+---------------+



In [12]:
# Calcolare la somma di tutti gli stipendi, reparto per reparto
totale_stipendi_reparti = dipendenti_data_raw.groupBy("Reparto").agg(F.sum("Stipendio").alias("Totale Stipendio per Reparto"))
totale_stipendi_reparti.show()

+-------+----------------------------+
|Reparto|Totale Stipendio per Reparto|
+-------+----------------------------+
|     HR|                        6400|
|    CEO|                        3000|
|     IT|                        5300|
+-------+----------------------------+



In [13]:
# Totale Stipendi Over Partition
totale_stipendio_over_partition = dipendenti_data_raw.withColumn("Totale Stipendi Reparto", F.sum("Stipendio").over(W.partitionBy("Reparto"))).drop("NomeCognome")
totale_stipendio_over_partition.show()

+--------+--------+-------+---------+-----------------------+
|    Nome| Cognome|Reparto|Stipendio|Totale Stipendi Reparto|
+--------+--------+-------+---------+-----------------------+
|   Ivana|  Itteri|    CEO|     3000|                   3000|
|   Dario|  Desini|     HR|     1200|                   6400|
|   Elena| Ettorre|     HR|     1600|                   6400|
|Federica| Fierini|     HR|     2200|                   6400|
|  Giulia|Giorgini|     HR|     1400|                   6400|
|  Andrea|  Assini|     IT|     1500|                   5300|
|   Bruno| Bettini|     IT|     2000|                   5300|
|   Carlo|Cicerini|     IT|     1800|                   5300|
+--------+--------+-------+---------+-----------------------+



In [14]:
# Totale Stipendi Over Partition Order By
totale_stipendio_over_partition = dipendenti_data_raw.withColumn("Totale Stipendi Reparto", F.sum("Stipendio").over(W.partitionBy("Reparto").orderBy("Nome"))).drop("NomeCognome")
totale_stipendio_over_partition.show()

+--------+--------+-------+---------+-----------------------+
|    Nome| Cognome|Reparto|Stipendio|Totale Stipendi Reparto|
+--------+--------+-------+---------+-----------------------+
|   Ivana|  Itteri|    CEO|     3000|                   3000|
|   Dario|  Desini|     HR|     1200|                   1200|
|   Elena| Ettorre|     HR|     1600|                   2800|
|Federica| Fierini|     HR|     2200|                   5000|
|  Giulia|Giorgini|     HR|     1400|                   6400|
|  Andrea|  Assini|     IT|     1500|                   1500|
|   Bruno| Bettini|     IT|     2000|                   3500|
|   Carlo|Cicerini|     IT|     1800|                   5300|
+--------+--------+-------+---------+-----------------------+

